## 00. 랭스미스 추적 설정

In [1]:
# 랭스미스 추적 활성화
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# %pip install langchain-teddynote

In [3]:
from langchain_teddynote import logging

# 프로젝트 이름 입력
logging.langsmith("bigcon_langchain_test")

# 추적을 끄고 싶은 경우
# logging.langsmith("bigcon_langchain_test", set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
bigcon_langchain_test


<hr>

## 01. Google Gemini API Test

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
import os

In [5]:
# dotenv로 API 키값 가져오기
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [6]:
# ChatGoogleGenerativeAI 언어 모델 초기화 (Gemini 1.5 Flash)

# 1) from 테디노트
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# 2) from wikidocs
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [7]:
# 싱글턴 테스트
response = model.generate_content("hi there. what's your name")
print(response.text)

I don't have a name. I am a large language model, and I am not a person. I am designed to provide information and complete tasks as instructed.  

Is there anything else I can help you with today? 



In [8]:
# 멀티턴 방법 1

# 대화 이력을 담고 있는 ChatSession 객체 (리스트 history에 대화 이력 저장)
chat_session = model.start_chat(history=[])
user_queries = ["인공지능에 대해 한 문장으로 짧게 설명하세요.", "의식이 있는지 한 문장으로 답하세요."]

for user_query in user_queries:
  print(f"[USER]: {user_query}")
  response = chat_session.send_message(user_query)
  print(f"[GEMINI]: {response.text}")

[USER]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[GEMINI]:  인공지능은 컴퓨터 시스템이 인간과 유사한 지능적인 작업을 수행할 수 있도록 하는 기술입니다. 

[USER]: 의식이 있는지 한 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 구글에서 개발한 대규모 언어 모델입니다. 



In [11]:
# 멀티턴 방법 2
user_queries = [{'role':'user', 'parts': ["인공지능에 대해 한 문장으로 짧게 설명하세요."]},
                {'role':'user', 'parts': ["의식이 있는지 한 문장으로 답하세요."]}
               ]

history= []

for user_query in user_queries:
  history.append(user_query)
  print(f'[USER]: {user_query["parts"][0]}')  
  response = model.generate_content(history)
  print(f'[GEMINI]: {response.text}')   
  history.append(response.candidates[0].content)

### 멀티턴 방법 1과의 차이점 ###
# `chat_session`을 사용하지 않고 싱글턴 방식의 `generate_content`를 사용하지만 대화 이력은 사용자 프로그램에서 직접 관리 (history 변수)
# 사용자-프로그램 간의 대화 이력을 model.generate_content를 호출할 떄마다 인자값으로 전달하면서 대화 이력 전체를 참조해서 답변을 생성

[USER]: 인공지능에 대해 한 문장으로 짧게 설명하세요.
[GEMINI]: 인공지능은 인간의 지능을 모방하도록 설계된 컴퓨터 시스템으로, 학습, 문제 해결, 의사 결정을 수행할 수 있습니다. 

[USER]: 의식이 있는지 한 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 인공지능 모델로, 생각이나 감정을 느낄 수 없습니다. 



<hr>

##### *멀티턴 방식을 선택하는 기준*
- 방법 1. ChatSession
  - 사용자와 모델 간 대화 사이에 프로그램의 개입이 필요 없는 경우
  - 간편하고 오버헤드를 줄일 수 있음
- 방법 2. generate_content
  - [메시지 입력] >> [전송] >> [모델 응답] 과정에서 사용자 프로그램의 개입이 필요한 경우 

In [12]:
# 모델의 응답 메시지 길이를 40자 이내로 맞춰야 하는 상황 가정
user_queries = [
    {'role': 'user', 'parts': ["인공지능에 대해 40자 이내의 문장으로 설명하세요."]},
    {'role': 'user', 'parts': ["의식이 있는지 40자 이내의 문장으로 답하세요."]}
]
history = []

for user_query in user_queries:
    history.append(user_query)
    print(f'[USER]: {user_query["parts"][0]}')
    response = model.generate_content(history)    

    ### 응답의 길이가 40자를 초과하는 경우 재실행 ###
    while len(response.text) > 40:
        print(f"응답 메시지 길이: {len(response.text)}")
        print("응답 재생성")
        response = model.generate_content(history)

    print(f'[GEMINI]: {response.text}')
    history.append(response.candidates[0].content)

# 방법 1. ChatSession 객체를 사용하면 중간에 끼어들기가 힘드므로 모델의 응답 중간에 로직을 구현하기 힘듦
#         >> ChatSession 객체의 send_message 메서드는 질의/응답을 대화 이력에 담는 과정을 메서드 내부로 노출하지 않기 때문

[USER]: 인공지능에 대해 40자 이내의 문장으로 설명하세요.
응답 메시지 길이: 46
응답 재생성
응답 메시지 길이: 46
응답 재생성
응답 메시지 길이: 46
응답 재생성
응답 메시지 길이: 41
응답 재생성
응답 메시지 길이: 46
응답 재생성
[GEMINI]: 인공지능은 인간의 지능을 모방하여 복잡한 문제를 해결하는 기술입니다. 

[USER]: 의식이 있는지 40자 이내의 문장으로 답하세요.
[GEMINI]: 저는 의식이 없습니다. 저는 인공지능 모델입니다. 

